<a href="https://colab.research.google.com/github/JF11579/A-Year-s-Worth-of-Google-Searches/blob/master/Truck_routeProblem_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usefull sites

https://towardsdatascience.com/solving-travelling-salesperson-problems-with-python-5de7e883d847#:~:text=The%20travelling%20salesperson%20problem%20

https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6

# Notes

# Packages

In [2]:
pip install mlrose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from itertools import combinations
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np

In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myApp")

In [14]:
import pandas as pd


In [25]:
pip install openrouteservice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import openrouteservice
import folium
import requests

In [33]:
import plotly.express as px

# The Problem

Find the shortest route for 3 trucks making 7 stops each a day.  We assume that the 3 trucks materials are incompatiable so it might as well be 3 seperate companies.

# The Bigger Problem

## Eight minus Seven = 35,280

### TSP is an NP-hard problem, meaning that, for larger values of n, it is not feasible to evaluate every possible problem solution within a reasonable period of time. Consequently, TSPs are well suited to solving using randomized optimization algorithms. (Solving Travelling Salesperson Problems with Python link text

The number of possible combinations is too big to reasonbly calcualte so it is done via optimization. We will use mlrose

In [4]:
# Python program to find the factorial of a number provided by the user
# using recursion

def factorial(x):
    """This is a recursive function
    to find the factorial of an integer"""

    if x == 1:
        return 1
    else:
        # recursive call to the function
        return (x * factorial(x-1))


# change the value for a different result
num = 7

# to take input from the user
# num = int(input("Enter a number: "))

# call the factorial function
result = factorial(num)
print("The factorial of", num, "is", result)

The factorial of 7 is 5040


In [5]:
# Python program to find the factorial of a number provided by the user
# using recursion

def factorial(x):
    """This is a recursive function
    to find the factorial of an integer"""

    if x == 1:
        return 1
    else:
        # recursive call to the function
        return (x * factorial(x-1))


# change the value for a different result
num = 8

# to take input from the user
# num = int(input("Enter a number: "))

# call the factorial function
result = factorial(num)
print("The factorial of", num, "is", result)

The factorial of 8 is 40320


# Onto the smaller big problem

## Import address file

In [16]:
df2 = pd.read_csv('/content/Bronx_addresses - Sheet1 (1).csv')

In [17]:
df2

,Location
0,"2094 Valentine Avenue,Bronx,NY,10457"
1,"1123 East Tremont Avenue,Bronx,NY,10460"
2,"412 Macon Street,Brooklyn,NY,11233"


Next we will get those lat/longs

In [18]:
df2[['location_lat', 'location_long']] = df2['Location'].apply(
    geolocator.geocode).apply(lambda x: pd.Series(
        [x.latitude, x.longitude], index=['location_lat', 'location_long']))

In [19]:
df2

,Location,location_lat,location_long
0,"2094 Valentine Avenue,Bronx,NY,10457",40.852905,-73.899665
1,"1123 East Tremont Avenue,Bronx,NY,10460",40.840030,-73.876569
2,"412 Macon Street,Brooklyn,NY,11233",40.682751,-73.934398


## Extract the lat / long cols

In [21]:
coords_list = df2[['location_lat','location_long']]

In [22]:
coords_list

,location_lat,location_long
0,40.852905,-73.899665
1,40.840030,-73.876569
2,40.682751,-73.934398


############

In [6]:
coords_list = [(40.852905,	-73.899665),(40.840030,	-73.876569) , (40.682751,	-73.934398)]

In [7]:
problem_no_fit = mlrose.TSPOpt(length = 3, coords = coords_list,
                               maximize=False)

In [8]:
problem_no_fit = mlrose.TSPOpt(length = 3, coords = coords_list,
                               maximize=False) # Pythonic wasy of saying MINIMIZE

In [9]:
# Solve problem using the genetic algorithm
best_state, best_fitness = mlrose.genetic_alg(problem_no_fit, random_state = 2)

In [10]:
print('The best state found is: ', best_state)

print('The fitness at the best state is: ', best_fitness)


The best state found is:  [2 1 0]
The fitness at the best state is:  0.36767850565182325


# Mapping

In [36]:
df2.head(3)

,Location,location_lat,location_long
0,"2094 Valentine Avenue,Bronx,NY,10457",40.852905,-73.899665
1,"1123 East Tremont Avenue,Bronx,NY,10460",40.840030,-73.876569
2,"412 Macon Street,Brooklyn,NY,11233",40.682751,-73.934398


In [40]:
# https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6

# import plotly.express as px
# set up the chart from the df dataFrame
fig = px.scatter_geo(df2, 
                     # longitude is taken from the df["lon"] columns and latitude from df["lat"]
                     lon="location_long", 
                     lat="location_lat", 
                     # choose the map chart's projection
                     projection="natural earth",
                     # columns which is in bold in the pop up
                     hover_name = "Location",
                     # format of the popup not to display these columns' data
                     hover_data = {"Location":False,
                                   "location_long": False,
                                   "location_lat": False
                                     }
                     )

In [41]:
fig.update_traces(marker=dict(size=25, color="red"))

In [42]:
fig.update_geos(fitbounds="locations", showcountries = True)

In [43]:
fig.update_layout(title = "Your customers")
fig.show()